### Hash Maps Coding Exercises Set 2

Consider the following classes:

In [1]:
class Stock:
    def __init__(self, symbol, date, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume

#### Exercise 1

Given the above class, write a custom JSONEncoder class to serialize dictionaries that contain instances of these particular classes. Keep in mind that you will want to deserialize the data too- so you will need some tehcnique to indicate the object type in your serialization.

For example, you may have an object such as this one that needs to be serialized:

In [2]:
from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22),
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22),
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22),
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689),
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 5, 12), 'sell', Decimal('177.01'), 20, Decimal('9.99')),
    ]
}

Hint: You can modufy the classes if you need to!

#### Solution

In [3]:
class Stock:
    def __init__(self, symbol, date, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
    def as_dict(self):
        return dict(symbol=self.symbol,
                    date=self.date,
                    open=self.open,
                    high=self.high,
                    low=self.low,
                    close=self.close,
                    volume=self.volume
                   )
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume
        
        
    def as_dict(self):
        return dict(symbol=self.symbol,
                    timestamp=self.timestamp,
                    order=self.order,
                    price=self.price,
                    commission=self.commission,
                    volume=self.volume
                   )

In [4]:
from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22),
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22),
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22),
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689),
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 5, 12), 'sell', Decimal('177.01'), 20, Decimal('9.99')),
    ]
}

In [5]:
from json import JSONEncoder, dumps

In [6]:
class CustomEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock):
            return obj.as_dict()
        elif isinstance(obj, Trade):
            return obj.as_dict()
        else:
            super().default(obj)

In [7]:
dumps(activity, cls=CustomEncoder)

TypeError: Object of type date is not JSON serializable

In [8]:
class CustomEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock):
            result = obj.as_dict()
            result['date'] = result['date'].strftime('%Y-%m-%d')
            return result
        elif isinstance(obj, Trade):
            result = obj.as_dict()
            result['timestamp'] = result['timestamp'].strftime('%Y-%m-%dT%H:%M%S')
            return result
        else:
            super().default(obj)

In [9]:
dumps(activity, cls=CustomEncoder)

TypeError: Object of type Decimal is not JSON serializable

In [10]:
class CustomEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock) or isinstance(obj, Trade):
            return obj.as_dict()
        elif isinstance(obj, date):
            return obj.strftime('%Y-%m-%d')
        elif isinstance(obj, datetime):
            return obj.strftime('%Y-%m-%dT%H:%M:%S')
        elif isinstance(obj, Decimal):
            return str(obj)
        else:
            super().default(obj)

In [11]:
encoded = dumps(activity, cls=CustomEncoder, indent=2)

In [12]:
print(encoded)

{
  "quotes": [
    {
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": 365607
    },
    {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184
    },
    {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689
    }
  ],
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22",
      "order": "buy",
      "price": "338.25",
      "commission": "9.99",
      "volume": 100
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22",
      "order": "sell",
      "price": "177.01",
      "commission": "9.99",
      "volume": 20
    }
  ]
}


In [13]:
class CustomEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Stock) or isinstance(obj, Trade):
            result = obj.as_dict()
            result['object'] = obj.__class__.__name__
            return result
        elif isinstance(obj, date):
            return obj.strftime('%Y-%m-%d')
        elif isinstance(obj, datetime):
            return obj.strftime('%Y-%m-%dT%H:%M:%S')
        elif isinstance(obj, Decimal):
            return str(obj)
        else:
            super().default(obj)

In [14]:
encoded = dumps(activity, cls=CustomEncoder, indent=2)

In [15]:
print(encoded)

{
  "quotes": [
    {
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": 365607,
      "object": "Stock"
    },
    {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184,
      "object": "Stock"
    },
    {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    }
  ],
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22",
      "order": "buy",
      "price": "338.25",
      "commission": "9.99",
      "volume": 100,
      "object": "Trade"
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22",
      "order": "sell",
      "price": "177.01",
      "commission": "9.99",
      "v

All done.

#### Excercise 2

Write code to reverse the serialization you just created. Write a custom decoder that can deserialize a JSON structure containing **Stock** and **Trade** objects.

#### Solution

In [16]:
def decode_stock(d):
    s = Stock(d['symbol'],
              datetime.strptime(d['date'], '%Y-%m-%d').date(),
              Decimal(d['open']),
              Decimal(d['high']),
              Decimal(d['low']),
              Decimal(d['close']),
              d['volume']
             )
    return s

In [17]:
d = {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184,
      "object": "Stock"
    }

In [18]:
s = decode_stock(d)

In [19]:
type(s)

__main__.Stock

In [20]:
vars(s)

{'symbol': 'AAPL',
 'date': datetime.date(2018, 11, 22),
 'open': Decimal('176.66'),
 'high': Decimal('177.25'),
 'low': Decimal('176.64'),
 'close': Decimal('176.78'),
 'volume': 3699184}

In [21]:
def decode_trade(d):
    s = Trade(d['symbol'],
              datetime.strptime(d['timestamp'], '%Y-%m-%d'),
              d['order'],
              Decimal(d['price']),
              int(d['volume']),
              Decimal(d['commission'])
             )
    return s

In [22]:
t = decode_trade({
      "symbol": "AAPL",
      "timestamp": "2018-11-22",
      "order": "sell",
      "price": "177.01",
      "commission": "9.99",
      "volume": 20,
      "object": "Trade"
    })

In [23]:
type(t)

__main__.Trade

In [24]:
vars(t)

{'symbol': 'AAPL',
 'timestamp': datetime.datetime(2018, 11, 22, 0, 0),
 'order': 'sell',
 'price': Decimal('177.01'),
 'commission': Decimal('9.99'),
 'volume': 20}

In [25]:
def decode_financials(d):
    object_type = d.get('object', None)
    if object_type == 'Stock':
        return decode_stock(d)
    elif object_type == 'Trade':
        return decode_trade(d)
    return d

In [26]:
decode_financials({
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:30:05",
      "order": "sell",
      "price": "177.01",
      "commission": "9.99",
      "volume": 20,
      "object": "Trade"
    })

ValueError: unconverted data remains: T10:30:05

In [27]:
decode_financials({
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184,
      "object": "Stock"
    })

In [28]:
decode_financials({'a': 1})

{'a': 1}

In [29]:
from json import JSONDecoder, loads

In [30]:
class CustomDecoder(JSONDecoder):
    def decode(self, arg):
        data = loads(arg)
        return self.parse_financials(data)
    
    def parse_financials(self, obj):
        if isinstance(obj, dict):
            obj = decode_financials(obj)
            if isinstance(obj, dict):
                for key, value in obj.items():
                    obj[key] = self.parse_financials(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.parse_financials(item)
        return obj

In [31]:
print(encoded)

{
  "quotes": [
    {
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": 365607,
      "object": "Stock"
    },
    {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": 3699184,
      "object": "Stock"
    },
    {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": 4493689,
      "object": "Stock"
    }
  ],
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22",
      "order": "buy",
      "price": "338.25",
      "commission": "9.99",
      "volume": 100,
      "object": "Trade"
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22",
      "order": "sell",
      "price": "177.01",
      "commission": "9.99",
      "v

In [32]:
decoded = loads(encoded, cls=CustomDecoder)

In [33]:
decoded

{'quotes': [<__main__.Stock at 0x1aad963cd08>,
 'trades': [<__main__.Trade at 0x1aad963c748>,
  <__main__.Trade at 0x1aad963c9c8>]}

In [34]:
decoded == activity

False

In [35]:
class Stock:
    def __init__(self, symbol, date, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        
    def as_dict(self):
        return dict(symbol=self.symbol,
                    date=self.date,
                    open=self.open,
                    high=self.high,
                    low=self.low,
                    close=self.close,
                    volume=self.volume
                   )
    
    def __eq__(self, other):
        return isinstance(other, Stock) and self.as_dict() == other.as_dict()
        
class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume
        
        
    def as_dict(self):
        return dict(symbol=self.symbol,
                    timestamp=self.timestamp,
                    order=self.order,
                    price=self.price,
                    commission=self.commission,
                    volume=self.volume
                   )
    
    def __eq__(self, other):
        return isinstance(other, Trade) and self.as_dict() == other.as_dict()

In [36]:
activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22),
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22),
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22),
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689),
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 5, 12), 'sell', Decimal('177.01'), 20, Decimal('9.99')),
    ]
}

In [37]:
encoded = dumps(activity, cls=CustomEncoder, indent=2)
decoded = loads(encoded, cls=CustomDecoder)

In [38]:
activity == decoded

False

This should be true, but there's a glitch because of the timestamp being a date object.

#### Exercise 3

Do the same serialization and deserialization, but using **Marshmallow**.

#### Solution

In [39]:
from marshmallow import Schema, fields

In [40]:
class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal()
    high = fields.Decimal()
    low = fields.Decimal()
    close = fields.Decimal()
    volume = fields.Integer()

In [42]:
StockSchema().dump(Stock('TSLA', date(2018, 11, 22),
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607))

{'close': Decimal('338.19'),
 'volume': 365607,
 'open': Decimal('338.19'),
 'date': '2018-11-22',
 'high': Decimal('338.64'),
 'symbol': 'TSLA',
 'low': Decimal('337.60')}

In [43]:
StockSchema().dumps(Stock('TSLA', date(2018, 11, 22),
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607))

TypeError: Object of type Decimal is not JSON serializable

In [48]:
class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal(as_string=True)
    high = fields.Decimal(as_string=True)
    low = fields.Decimal(as_string=True)
    close = fields.Decimal(as_string=True)
    volume = fields.Integer()

In [49]:
StockSchema().dumps(Stock('TSLA', date(2018, 11, 22),
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607))

'{"close": "338.19", "volume": 365607, "open": "338.19", "date": "2018-11-22", "high": "338.64", "symbol": "TSLA", "low": "337.60"}'

In [74]:
class TradeSchema(Schema):
    symbol = fields.Str()
    timestamp = fields.DateTime()
    order = fields.Str()
    price = fields.Decimal(as_string=True)
    commission = fields.Decimal(as_string=True)
    volume = fields.Integer()

In [75]:
TradeSchema().dumps(Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')))

'{"volume": 100, "symbol": "TSLA", "timestamp": "2018-11-22T10:05:12", "commission": "9.99", "order": "buy", "price": "338.25"}'

In [76]:
class ActivitySchema(Schema):
    quotes = fields.Nested(StockSchema, many=True)
    trades = fields.Nested(TradeSchema, many=True)

In [77]:
result = ActivitySchema().dumps(activity, indent=2)

In [78]:
type(result)

str

In [79]:
print(result)

{
  "quotes": [
    {
      "close": "338.19",
      "volume": 365607,
      "open": "338.19",
      "date": "2018-11-22",
      "high": "338.64",
      "symbol": "TSLA",
      "low": "337.60"
    },
    {
      "close": "176.78",
      "volume": 3699184,
      "open": "176.66",
      "date": "2018-11-22",
      "high": "177.25",
      "symbol": "AAPL",
      "low": "176.64"
    },
    {
      "close": "103.11",
      "volume": 4493689,
      "open": "103.25",
      "date": "2018-11-22",
      "high": "103.48",
      "symbol": "MSFT",
      "low": "103.07"
    }
  ],
  "trades": [
    {
      "volume": 100,
      "symbol": "TSLA",
      "timestamp": "2018-11-22T10:05:12",
      "commission": "9.99",
      "order": "buy",
      "price": "338.25"
    },
    {
      "volume": 20,
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:05:12",
      "commission": "9.99",
      "order": "sell",
      "price": "177.01"
    }
  ]
}


In [80]:
activity_deser = ActivitySchema().loads(result)

In [81]:
type(activity_deser)

dict

In [63]:
from pprint import pprint
pprint(activity_deser)

{'quotes': [{'close': Decimal('338.19'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('338.64'),
             'low': Decimal('337.60'),
             'open': Decimal('338.19'),
             'symbol': 'TSLA',
             'volume': 365607},
            {'close': Decimal('176.78'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('177.25'),
             'low': Decimal('176.64'),
             'open': Decimal('176.66'),
             'symbol': 'AAPL',
             'volume': 3699184},
            {'close': Decimal('103.11'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('103.48'),
             'low': Decimal('103.07'),
             'open': Decimal('103.25'),
             'symbol': 'MSFT',
             'volume': 4493689}],
 'trades': [{'commission': Decimal('9.99'),
             'order': 'buy',
             'price': Decimal('338.25'),
             'symbol': 'TSLA',
             'timestamp': datetim

In [65]:
from marshmallow import post_load

class TradeSchema(Schema):
    symbol = fields.Str()
    timestamp = fields.DateTime()
    order = fields.Str()
    price = fields.Decimal(as_string=True)
    commission = fields.Decimal(as_string=True)
    volume = fields.Integer()
    
    @post_load
    def make_trade(self, data):
        return Trade(**data)
    
class ActivitySchema(Schema):
    quotes = fields.Nested(StockSchema, many=True)
    trades = fields.Nested(TradeSchema, many=True)

In [66]:
result = ActivitySchema().dumps(activity, indent=2)
print(result)

{
  "quotes": [
    {
      "close": "338.19",
      "volume": 365607,
      "open": "338.19",
      "date": "2018-11-22",
      "high": "338.64",
      "symbol": "TSLA",
      "low": "337.60"
    },
    {
      "close": "176.78",
      "volume": 3699184,
      "open": "176.66",
      "date": "2018-11-22",
      "high": "177.25",
      "symbol": "AAPL",
      "low": "176.64"
    },
    {
      "close": "103.11",
      "volume": 4493689,
      "open": "103.25",
      "date": "2018-11-22",
      "high": "103.48",
      "symbol": "MSFT",
      "low": "103.07"
    }
  ],
  "trades": [
    {
      "volume": 100,
      "symbol": "TSLA",
      "timestamp": "2018-11-22T10:05:12",
      "commission": "9.99",
      "order": "buy",
      "price": "338.25"
    },
    {
      "volume": 20,
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:05:12",
      "commission": "9.99",
      "order": "sell",
      "price": "177.01"
    }
  ]
}


In [82]:
activity_deser = ActivitySchema().loads(result)

Same issue as before in lecture with Marshmallow having changed since uploading of videos... no big deal